In [1]:
import numpy as np
import pandas as pd
import time
import datetime

In [3]:
#Install the dependencies
# import quandl
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [31]:
import pandas as pd
df = pd.read_csv('Data/FB_data.csv')
df.index = df['Date']
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2014-12-31,2014-12-31,79.540001,79.800003,77.860001,78.019997,78.019997,19935400
2015-01-02,2015-01-02,78.580002,78.930000,77.699997,78.449997,78.449997,18177500
2015-01-05,2015-01-05,77.980003,79.250000,76.860001,77.190002,77.190002,26452200
2015-01-06,2015-01-06,77.230003,77.589996,75.360001,76.150002,76.150002,27399300
2015-01-07,2015-01-07,76.760002,77.360001,75.820000,76.150002,76.150002,22045300


In [32]:
# Get the Adjusted Close Price 
df = df.drop(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], axis=1)
# Take a look at the new data 
print(df.head())

            Adj Close
Date                 
2014-12-31  78.019997
2015-01-02  78.449997
2015-01-05  77.190002
2015-01-06  76.150002
2015-01-07  76.150002


In [33]:
# A variable for predicting 'n' days out into the future
forecast_out = 30 #'n=30' days
#Create another column (the target ) shifted 'n' units up
df['Prediction'] = df['Adj Close'].shift(-forecast_out)
#print the new data set
print(df.tail())

             Adj Close  Prediction
Date                              
2019-10-09  179.850006         NaN
2019-10-10  180.029999         NaN
2019-10-11  184.190002         NaN
2019-10-14  183.279999         NaN
2019-10-15  188.889999         NaN


In [34]:
### Create the independent data set (X)  #######
# Convert the dataframe to a numpy array
X = np.array(df.drop(['Prediction'],1))

#Remove the last '30' rows
X = X[:-forecast_out]
print(X)

[[ 78.019997]
 [ 78.449997]
 [ 77.190002]
 ...
 [185.570007]
 [185.669998]
 [182.389999]]


In [35]:
### Create the dependent data set (y)  #####
# Convert the dataframe to a numpy array 
y = np.array(df['Prediction'])
# Get all of the y values except the last '30' rows
y = y[:-forecast_out]
print(y)

[ 75.739998  75.599998  76.709999 ... 184.190002 183.279999 188.889999]


In [36]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [37]:
# Create and train the Support Vector Machine (Regressor) 
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1) 
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [38]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[:]
print(x_forecast)

[[ 78.019997]
 [ 78.449997]
 [ 77.190002]
 ...
 [184.190002]
 [183.279999]
 [188.889999]]


In [39]:
# Print support vector regressor model predictions for the next '30' days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[ 81.98984385  81.55919902  82.17968826 ... 179.11775163 179.6633666
 190.55838534]


In [40]:
df['Prediction'] = svm_prediction

In [41]:
df

,Adj Close,Prediction
Date,,
2014-12-31,78.019997,81.989844
2015-01-02,78.449997,81.559199
2015-01-05,77.190002,82.179688
2015-01-06,76.150002,80.684085
2015-01-07,76.150002,80.684085
2015-01-08,78.180000,81.821747
2015-01-09,77.739998,82.222941
2015-01-12,76.720001,81.577523
2015-01-13,76.449997,81.124305


In [42]:
import pickle
with open('fb_model.pkl', 'wb') as f:
    pickle.dump(svr_rbf, f)